In [ ]:
#default_exp runner

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import blocks
import logging
import os
import tempfile
import uuid
import warnings
import yaml
import pickle

from io import TextIOWrapper
from datetime import datetime
from functools import lru_cache
from pathlib import Path
from blocks.filesystem import GCSFileSystem as gcsfs
from googleapiclient import discovery
from google.cloud.logging.handlers.handlers import CloudLoggingHandler, EXCLUDED_LOGGER_DEFAULTS
import google.cloud.logging

from yoda import tags

# Shared

## Data class

For keeping track of all data related objects and don't reload data.

In [ ]:
#export
class Data:
    def __init__(self,
                 input_path: str,
                 output_path: str,
                 features: str,
                 label: str,
                 **kwargs):
        self.input_path = input_path
        self.output_path = output_path
        self.feature_list = features.split(",")
        self.label = label
        self.eval_path = kwargs.get("eval_path", None)
        self.score_path = kwargs.get("score_path", None)
        self.is_gcp = input_path.startswith("gs://")

    @property
    @lru_cache(1)
    def df(self):
        return blocks.assemble(self.input_path)

    @property
    def X(self):
        return self.df[self.feature_list]

    @property
    def y(self):
        return self.df[self.label]

    @property
    @lru_cache(1)
    def eval_df(self):
        if not self.eval_path:
            raise Exception("Please specify the eval_path")
        return blocks.assemble(self.eval_path)

    @property
    def eval_X(self):
        return self.eval_df[self.feature_list]

    @property
    def eval_y(self):
        return self.eval_df[self.label]

    @property
    @lru_cache(1)
    def score_df(self):
        if not self.score_path:
            raise Exception("Please specify the score_path")
        return blocks.assemble(self.score_path)

    @property
    def score_X(self):
        return self.score_df[self.feature_list]

    def open(self, filename) -> TextIOWrapper:
        full_path = os.path.join(self.output_path, filename)
        opener = gcsfs().open if self.is_gcp else open
        with opener(full_path) as fobj:
            yield fobj

## Model

In [ ]:
#export
def _import_from_string(classname: str):
    components = classname.split('.')
    mod = __import__(components[0])
    for comp in components[1:]:
        mod = getattr(mod, comp)
    return mod


def _parse_model_dict(model_dict: dict):
    if "model_code" in model_dict:
        global_dict = dict()
        exec(model_dict["model_code"], global_dict)
        model_class = global_dict[model_dict["estimator"]]
    else:
        model_class = _import_from_string(model_dict["estimator"])

    def save(self, fileobj):
        pickle.dump(self, fileobj)

    setattr(model_class, "save", save)
    return model_class

In [ ]:
model_conf = {"estimator": "xgboost.XGBClassifier"}

_parse_model_dict(model_conf)

xgboost.sklearn.XGBClassifier

In [ ]:
model_conf = {"model_code": open("../data/models/LGBDefault.py").read(),
              "estimator": "LGBExt"}

_parse_model_dict(model_conf)

LGBExt

## Evaluation

In [ ]:
#export
def _eval(estimator: sklearn.base.BaseEstimator = None,
          data: Data = None,
          cv=None,
          metrics: str = None) -> dict:
    if data.eval_path is not None and cv is not None:
        raise Exception(
            "eval_path: (%s) and cv: (%s) cannot co-exist" % (data.eval_path, cv))

    eval_res = dict()
    for metric in metrics:
        if data.eval_path is not None:
            estimator.fit(data.X, data.y)
            scorer = sklearn.metrics.SCORERS[metric]
            avg, sd = scorer(estimator, data.eval_X, data.eval_y), 0
        if cv is not None:
            scores = cross_val_score(
                estimator, data.X, data.y, cv=cv, scoring=metric)
            avg, sd = np.mean(scores), np.std(scores)

        eval_res[metric] = {"sd": sd, "avg": avg}
    return eval_res


def run_eval(conf_dict: dict, data: Data, estimator, output_dir: str = "eval.pkl"):
    eval_path = data.eval_path
    metrics_str = conf_dict["eval"].get("metrics")
    cv = conf_dict["eval"].get("cv")
    metrics = metrics_str.split(",") if metrics_str else None
    result = _eval(estimator, data, cv, metrics)
    conf_cp = dict(conf_dict)
    conf_cp["eval_result"] = result
    # TODO: consider to create an class here and do
    # evaluate.save()
    if output_dir:
        pickle.dump(conf_cp, data.open(output_dir))
    return result

# Run on local

In [ ]:
#export
def run_on_local(conf_dict: dict):
    data = Data(**conf_dict['data'])
    model = _parse_model_dict(conf_dict['model'])
    if "eval" in conf_dict or data.eval_path:
        run_eval(conf_dict, data, train.estimator)
    else:
        train.fit(data.X, data.y)
        train.save(data.open("model.pkl"))

In [ ]:
config1 = '../data/configs/config1.yaml'
with open(config1) as f:
    print(f.read())

conf = yaml.safe_load(open(config1))

data: 
  input_df: !data "../data/iris_data.csv"
  eval_df: !data "../data/iris_data.csv"
  output_path: "../output/"
  features: "sepal_length,sepal_width,petal_length"
  label: species
model:
  estimator: xgboost.XGBClassifier
  params:
    max_depth: 4
    num_estimator: 50
eval:
  metrics: "accuracy,f1_macro"


NameError: name 'os' is not defined

# Run on GCP

In [ ]:
#export
class GCPConfig:
    @staticmethod
    @lru_cache(1)
    def bucket():
        return os.getenv("BUCKET") or input("Please enter the bucket path: ").strip()

    @staticmethod
    @lru_cache(1)
    def project_id():
        return os.getenv("PROJECT_ID") or google.auth.default()[1] or input("Please enter the project id: ").strip()


In [ ]:
#export
def _setup_logging():
    job = os.environ.get("CLOUD_ML_JOB_ID", None)
    trial = os.environ.get("CLOUD_ML_TRIAL_ID", None)
    project = os.environ.get("GCP_PROJECT", None)
    if job and project:
        client = google.cloud.logging.Client(project = project)
        resource = Resource(type = "ml_job", labels = dict(job_id = job, project_id = project, task_name = "master-replica-o"))
        # grouping by trial in AIP logs
        labels = {"ml.googleapis.com/trial_id": trial} if trial is not None else None
        handler = CloudLoggingHandler(client, resource=resource,labels=labels)
        logger = logging.getLogger()
        logger.handlers = []
        logger.setLevel(logging.DEBUG)
        logger.addHandler(handler)
        for logger_name in EXCLUDED_LOGGER_DEFAULTS:
            logging.getLogger(logger_name).propagate = False
    else:
        logger = logging.getLogger()
        logger.setLevel(logging.DEBUG)

In [ ]:
#export
class AIP:

    @property
    @lru_cache(1)
    def job_id(self):
        now = datetime.now()
        date_time = now.strftime("%Y%m%d_%H%M%S")
        gen_uunid = hex(uuid.getnode())
        return f"ai_run_{date_time}_{gen_uunid}"

    @property
    @lru_cache(1)
    def job_output(self):
        return GCPConfig.bucket()

    def run(self, image: str, machine_type: str = "n1-highmem-32", args=[], **overrides):
        """

        """
        # TODO: dealing with hyperparameters

        training_inputs = {
            "scaleTier": "CUSTOM",
            "masterType": machine_type,
            "args": args,
            "region": "us-central1",
            "masterConfig": {"imageUri": image}
        }
        training_inputs.update(overrides)

        job_spec = {"jobId": self.job_id, "trainingInput": training_inputs}
        project_id = "projects/{}".format(GCPConfig.project_id())

        _setup_logging()
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            cloudml = discovery.build("ml", "v1", cache_discovery=False)
            request = cloudml.projects().jobs().create(body=job_spec, parent=project_id)
            try:
                return request.execute()
            except e:
                logging.warn("You may want to check whether the image is in registery")
                raise e

In [ ]:
import os
import yaml

with open(r'../credentials.yaml') as file:
    creds = yaml.load(file, Loader=yaml.FullLoader)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = creds["google_credentials"]
image_url = creds["image_url"]

AIP().run(image_url)

DEBUG:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/ml/v1/rest
DEBUG:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/wmt-customer-tech-case-sci-dev/jobs?alt=json
DEBUG:google_auth_httplib2:Making request: POST https://oauth2.googleapis.com/token


{'jobId': 'ai_run_20200630_230635_0xacde48001122',
 'trainingInput': {'scaleTier': 'CUSTOM',
  'masterType': 'n1-highmem-32',
  'region': 'us-central1',
  'masterConfig': {'imageUri': 'gcr.io/wmt-customer-tech-case-sci-dev/yoda'}},
 'createTime': '2020-07-01T06:06:38Z',
 'state': 'QUEUED',
 'trainingOutput': {},
 'etag': 'Tvri1exxjCw='}

In [ ]:
#export
def _must_exist(key, dict_):
    assert key in dict_, "%r should be in the dictionary" % key


def _validate_config(conf_dict):
    _must_exist("image", conf_dict)
    _must_exist("data", conf_dict)
    _must_exist("output_path", conf_dict["data"])


def run_on_gcp(conf_dict):
    # validate config
    _validate_config(conf_dict)

    # upload config to gcp
    gcs_config_path = os.path.join(conf_dict["data"]["output_path"], "config.yaml")
    with gcsfs().open(gcs_config_path, "w") as f:
        yaml.safe_dump(conf_dict, f)

    # run on GCP
    args = "yoda run {}".format(gcs_config_path).split()
    image = conf_dict["image"]
    aip = AIP()
    aip.run(image, args=args)